# Extended data

In [50]:
import os
from pathlib import Path
import re
from datetime import datetime
from configparser import ConfigParser
import psycopg2
from psycopg2.extras import DictCursor
from psycopg2.extensions import AsIs
from docx import Document
from docx.shared import Pt,Mm
    
#from docx.shared import Inches

In [21]:
workdir = Path(os.path.expanduser('~')) / 'tmp' / 'typology-web-update-content'


In [3]:
filename = Path(os.path.expanduser('~')) / ".database.ini"
section = 'psqlaws'

parser = ConfigParser()
parser.read(filename)
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

params = db

In [125]:
conn = psycopg2.connect(**params)
cur = conn.cursor(cursor_factory=DictCursor)


In [146]:
qry="""
SELECT code, f.biome_code as biome, f.name, f.shortname, b.name as biome_name,
    realms, f.update as original_date, shortdesc, keyfeatures, distdesc 
FROM functional_groups f
LEFT JOIN biomes b
    ON b.biome_code=f.biome_code
WHERE code like '%s%%'
ORDER BY biome, REPLACE(code,f.biome_code||'.','')::int
;"""
efgs=dict()
for grp in ("M","F","S","T"):
    cur.execute(qry % grp)
    efgs[grp] = cur.fetchall()



In [147]:
len(efgs["F"])+len(efgs["T"])

66

In [78]:
mrg = 6000
fntsz=5
fntnm='Arial' # or Helvetica
pgwdth=Mm(183)
pghght=Mm(247)

document = Document()
sections = document.sections
section=sections[0]
section.left_margin=mrg
section.right_margin=mrg
section.top_margin=mrg
section.bottom_margin=mrg
section.page_width = pgwdth
section.page_height = pghght
thebiome=""

for grp in ("M","F","S","T"):
    for efg in efgs[grp]:
        if efg['biome_name'] != thebiome:
            thebiome=efg['biome_name']
            p = document.add_paragraph()
            paragraph_format = p.paragraph_format
            paragraph_format.space_before = Pt(0)
            paragraph_format.space_after = Pt(0)
            run = p.add_run(thebiome)
            font = run.font
            font.name = fntnm
            font.size = Pt(fntsz)
            font.bold = True
        p = document.add_paragraph()
        paragraph_format = p.paragraph_format
        paragraph_format.space_before = Pt(0)
        paragraph_format.space_after = Pt(0)
        run = p.add_run(efg['name'])
        font = run.font
        font.name = fntnm
        font.size = Pt(fntsz)
        font.bold = True
    
        run = p.add_run(": {}. {}.".format(efg['keyfeatures'],efg['distdesc']))
        font = run.font
        font.name = fntnm
        font.size = Pt(fntsz)
    if grp in ('M','F','S'):
        document.add_page_break()

document.save(workdir / 'Keith-et-al-ExtendedData-output-as-paragraphs.docx')

In [149]:
fntsz=3.5 # half size for output in one page
fntsz=7
def formattedCell(cell,txt,font_name=fntnm,font_size=fntsz,font_bold=False):
    ps=cell.paragraphs
    for p in ps:
        paragraph_format = p.paragraph_format
        paragraph_format.space_before = Pt(0)
        paragraph_format.space_after = Pt(0)
        run=p.add_run(txt)
        font = run.font
        font.name = font_name
        font.size = Pt(font_size)
        font.bold = font_bold    
    return(cell)

document = Document()
sections = document.sections
section=sections[0]
mrg = 6000
section.left_margin=mrg
section.right_margin=mrg
section.top_margin=mrg
section.bottom_margin=mrg

for grp in ("M","F","S","T"):
    if grp in ("M","F","S","T"): # or change to just M for output in one page
        table = document.add_table(rows=1, cols=3)
        hdr_cells = table.rows[0].cells
        formattedCell(hdr_cells[0],'Ecosystem Functional Group',font_bold=True)
        formattedCell(hdr_cells[1],'Typical Key features *',font_bold=True)
        formattedCell(hdr_cells[2],'Distribution',font_bold=True)

    for efg in efgs[grp]:
        if efg['biome_name'] != thebiome:
            thebiome=efg['biome_name']
            biome_cells =table.add_row().cells
            formattedCell(biome_cells[0],thebiome,font_bold=True)
            biome_cells[0].merge(biome_cells[2])
            #formattedCell(biome_cells[1],"-",font_bold=False)
            #formattedCell(biome_cells[2],"-",font_bold=False)
        row_cells = table.add_row().cells
        formattedCell(row_cells[0],efg['name'])
        formattedCell(row_cells[1],efg['keyfeatures'])
        formattedCell(row_cells[2],efg['distdesc'])
    if grp in ('M','F','S'): # or skip this for output in one page
        document.add_page_break()

document.save(workdir / 'Keith-et-al-ExtendedData-output-as-table.docx')

In [124]:
cur.close()
        
if conn is not None:
    conn.close()
    print('Database connection closed.')

Database connection closed.


Additional processing in `bash`:
